# More plots

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import statsmodels.stats.multitest

xs = np.linspace( -4, 4, 101 )
zero = 0 * xs
before = xs <= 0
after  = xs >= 0
y1 = scipy.stats.norm.pdf( xs, -1, 1 )
y2 = scipy.stats.norm.pdf( xs, 1, 1 )

fig, axs = plt.subplots( 1, 4, figsize = (12,1.8), layout = 'constrained' )
axs = axs.flatten()

axs[0].fill_between( xs[after], zero[after], y1[after], color = 'grey', alpha = 1 )
axs[0].fill_between( xs[after], zero[after], y2[after], color = 'grey', alpha = .3 )
axs[0].set_title( "FDR" )

axs[1].fill_between( xs[after], zero[after], y1[after], color = 'grey', alpha = 1 )
axs[1].fill_between( xs, zero, y1, color = 'grey', alpha = .3 )
axs[1].set_title( "False positive rate, alpha" )

axs[2].fill_between( xs[before], zero[before], y2[before], color = 'grey', alpha = 1 )
axs[2].fill_between( xs, zero, y2, color = 'grey', alpha = .3 )
axs[2].set_title( "beta" )

axs[3].fill_between( xs[after], zero[after], y2[after], color = 'grey', alpha = 1 )
axs[3].fill_between( xs, zero, y2, color = 'grey', alpha = .3 )
axs[3].set_title( "1-beta = power" )



for ax in axs: 
    ax.plot( xs, y1, linewidth = 3, label = 'H0' )
    ax.plot( xs, y2, linewidth = 3, label = 'H1' )
    ax.axvline( 0, color = 'black', linestyle = '--' )
    for side in ['left', 'right', 'top', 'bottom']:
        ax.spines[side].set_visible(False)
    ax.set_xticks([])
    ax.set_yticks([])
axs[0].legend( loc = 'upper left' )
plt.show()

In [ ]:
def test_fdr_benjamini_hochberg():

    # Example from the documentation        
    ps = [0.0001, 0.0004, 0.0019, 0.0095, 0.0201, 0.0278, 0.0298, 0.0344, 0.0459, 0.3240, 0.4262, 0.5719, 0.6528, 0.7590, 1.000]
    q = 0.05

    qs = scipy.stats.false_discovery_control(ps)
    #rejected, qs = statsmodels.stats.multitest.fdrcorrection(ps, q)  # Gives the same results

    i = np.argsort(ps)
    j = np.argsort(i)
    fig, ax = plt.subplots(figsize = (5,4), layout = 'constrained')
    ax.scatter( 
        np.arange( 1, len(ps)+1 ), ps, 
        marker = 'o', linestyle = '', 
        color = [ 'tab:red' if u < q else 'tab:blue' for u in qs ],
    )
    ax.axline( (0,0), slope = q / len(ps), linestyle = '--', color = 'black' )
    ax.set_xlabel( 'p-value rank' )
    ax.set_ylabel( 'p-value' )
    plt.show()

test_fdr_benjamini_hochberg()

In [ ]:
# Online FDR
# https://dsrobertson.github.io/onlineFDR/articles/theory.html
# There are other methods, using the number of rejected nulls, the time since the last rejected null, etc.

# Online FDR assumes streaming data, with P[H1] unknown; the paper assumes a batch of data, with P[H1] known.

alpha = 0.05
C = 0.07720838
js = np.arange(1, 101)
gammas = C * np.log( np.maximum(js,2) ) / ( js * np.exp( np.sqrt( np.log(js) ) ) )
fig, ax = plt.subplots( figsize = (12,3), layout = 'constrained' )
ax.plot( js, alpha * gammas, marker = 'o' )
ax.set_yscale('log')
ax.set_title( "Online FDR: LOND significance thresholds for alpha = 0.05 (multiply by 1 + the number of rejected nulls so far)" )
ax.set_ylabel( "Significance threshold" )
ax.set_xlabel( "Number of tests" )
plt.show()
 